# Introducción a las Redes Convolucionales

In [1]:
import keras
keras.__version__

/opt/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'2.1.6'

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

Vamos a comenzar mostrando una **Red Convolucional** muy simple para abordar el problema de clasificación MNIST que ya analizamos anteriormente. Con lo que hemos visto hasta ahora no resultará tan extraño, y más adelante detallaremos cada una de las capas que lo componen describiendo la funcionalidad que juegan en la red global. Veremos que, aunque la red convolucional que construiremos de forma directa es muy simple, supera el rendimiento de la red clásica que creamos en el primer notebook.

Si quieres leer una introducción (no matemática) de cómo y porqué funcionan las redes convolucionales puedes mirar [este magnífico blog de Ujjwal Karn](https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets).

![](CNN.png)

## Construyendo una CNN

Como se puede observar en el siguiente código, esencialmente la red convolucional que vamos a usar está formada por dos capas convoluciones bidimensionales `layer_conv_2d` seguidas de dos capas max_pooling `layer_max_pooling_2d`.

In [13]:
from keras import layers
from keras import models
from keras.utils.vis_utils import plot_model

CNN = models.Sequential()
CNN.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
CNN.add(layers.MaxPooling2D((2, 2)))
CNN.add(layers.Conv2D(64, (3, 3), activation='relu'))
CNN.add(layers.MaxPooling2D((2, 2)))
CNN.add(layers.Conv2D(64, (3, 3), activation='relu'))

plot_model(CNN, to_file='CNN_plot.png', show_shapes=True, show_layer_names=True)

Veamos un resumen de la arquitectura que hemos construido:
![](CNN_plot.png)

In [14]:
CNN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Una red convolucional toma como dato de entrada un tensor de la forma `(image_height, image_width, image_channels)`. En este caso, para ajustarse a las características de las imágenes de MNIST, será `(28, 28, 1)`, ya que usaremos un solo canal (gris) en las imágenes. Observa que el tamaño del dato de entrada se pasa como argumento de la primera capa con `input_shape = (28, 28, 1)`.

Cada capa de tipo `layer_conv_2d()` y `layer_max_pooling_2d()` dan como salida un tensor 3D de forma `(height, width, channels)`. Tanto la anchura como altura del tensor tienden a disminuir a medida que avanzamos en la red. El número de canales se controla por medio del primer argumento que se le pasa a las capas `layer_conv_2d()` (que en este caso es 32 o 64).

A continuación, hemos de pasar la salida de la última capa anterior (de tamaño `(3, 3, 64)`) a una red densa clasificadora similar a las que ya hemos visto en ejemplos anteriores. Como estas capas procesan vectores, que son 1D, y la entrada es un tensor 3D, hemos de aplanar el tensor por medio de la capa `layer_flatten()`, que también proporciona Keras:

In [5]:
CNN.add(layers.Flatten())
CNN.add(layers.Dense(64, activation='relu'))
CNN.add(layers.Dense(10, activation='softmax'))

plot_model(CNN, to_file='CNN2_plot.png', show_shapes=True, show_layer_names=True)

Como el objetivo es calcular una clasificación en 10 clases, la última capa es una capa densa con 10 unidades y con salida `softmax`. La red completa queda por tanto:

![](CNN2_plot.png)

In [6]:
CNN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

Una vez definida la red, realizamos el entrenamiento de forma similar a como hicimos en el modelo simple de MNIST:


In [7]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [12]:
CNN.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
CNN.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 4s 59us/step - loss: 0.0076 - acc: 0.9980
Epoch 2/5
60000/60000 [==============================] - 3s 54us/step - loss: 0.0051 - acc: 0.9987
Epoch 3/5
60000/60000 [==============================] - 3s 52us/step - loss: 0.0055 - acc: 0.9985
Epoch 4/5
60000/60000 [==============================] - 3s 56us/step - loss: 0.0048 - acc: 0.9988
Epoch 5/5
60000/60000 [==============================] - 3s 54us/step - loss: 0.0050 - acc: 0.9986


Tras el entrenamiento podemos evaluar el modelo sobre los datos de test:


In [9]:
test_loss, test_acc = CNN.evaluate(test_images, test_labels)

10000/10000 [==============================] - 0s 47us/step


In [10]:
test_acc

0.9898

Donde comprobamos que alcanzamos un accuracy de casi el 99%, un incremento relativo del 68% sobre el modelo básico visto anteriormente.

In [ ]:
#https://github.com/zalandoresearch/fashion-mnist